In [61]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [59]:
# --- Подготовка данных. Получение эксель-файла только с необходимыми столбцами ---

# Чтение данных с наименованиями полей
data_column_names = pd.read_excel('../input_data/data_from_site3.xlsx', sheet_name = 'names')

# Удаление событий сс пустыми значениями в поле checked_parameter
data_column_names['checked_parameter'].replace('', np.nan, inplace = True)
data_column_names.dropna(subset = ['checked_parameter'], inplace = True)

# Получение списка исследуемых параметров
list_data_necessary = []
for n_data in data_column_names['N']:
    list_data_necessary.append(n_data)
    
# Получение всех данных теста
data_all = pd.read_excel('../input_data/data_from_site2.xlsx', sheet_name = 'data')

# Получение данных по иследуемым параметрам
data_necessary = data_all[list_data_necessary]
data_necessary.head()

# Переименование заголовков столбцов
old_names = list_data_necessary
new_names = data_necessary.loc[0]
data_necessary.rename(columns = dict(zip(old_names, new_names)), inplace = True) 
data_necessary.drop([0], inplace = True)

# Создание столбца "Тип_занятости" взамен разрозненных данных
data_necessary['Тип_занятости'] = 0
data_necessary.loc[data_necessary['Владелец_предприятия'] == 1, 'Владелец_предприятия'] = 2
data_necessary.loc[data_necessary['ИП'] == 1, 'ИП'] = 3
data_necessary.loc[data_necessary['Фермер'] == 1, 'Фермер'] = 4
data_necessary.loc[data_necessary['Свободный_работник'] == 1, 'Свободный_работник'] = 5
data_necessary.loc[data_necessary['Мессенджеры_нет'] == '     .  ', 'Мессенджеры_нет'] = 99

for index, row in data_necessary.iterrows():
    if data_necessary['Наемный_работник'][index] == 1:
        data_necessary['Тип_занятости'][index] = data_necessary['Наемный_работник'][index]
    if data_necessary['Владелец_предприятия'][index] == 2:
        data_necessary['Тип_занятости'][index] = data_necessary['Владелец_предприятия'][index] 
    if data_necessary['ИП'][index] == 3:
        data_necessary['Тип_занятости'][index] = data_necessary['ИП'][index] 
    if data_necessary['Фермер'][index] == 4:
        data_necessary['Тип_занятости'][index] = data_necessary['Фермер'][index] 
    if data_necessary['Свободный_работник'][index] == 5:
        data_necessary['Тип_занятости'][index] = data_necessary['Свободный_работник'][index]  
    if data_necessary['Сумма_пенсии'][index] != '     .  ':
        data_necessary['Тип_занятости'][index] = 6
        data_necessary['Профессия_должность'][index] = 'пенсионер'
        data_necessary['Деятельность_компании'][index] = 'Пенсионный_Фонд_РФ'
        

drop_list = ['Наемный_работник', 'Владелец_предприятия', 'ИП', 'Фермер', 'Свободный_работник']        
data_necessary.drop(drop_list, axis = 'columns', inplace = True)  

# Сохранение отчасти подготовленных данных в файл
data_necessary.to_excel('../output_data/data_to_experiments.xlsx', sheet_name = 'data')

# # Проверка полученных данных по первым 5 значениям
# print(data_necessary.head())

# Удаление переменной для исключения конфликтов при повторном использовании
del data_necessary

# ---------------------------------------------------------------------------------

C:\Users\Tasha\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Tasha\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Tasha\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\User

Поскольку в полученных данных присутствуют суммы зарплаты и пенсии, то стоит выбор в методе обработки данных значений

1 вариант - нормализация столбцов, дополнение пустых столбцов по з/п данными пенсиями. По заполненным данным: среднее арифметическое? Или сумма без нормализации

2 вариант - сумму з/п и пенсии "=" уровень потребностей и коммуникаций, но не является логичным показателем субъективных ощущений.
(Т.к. низкий уровень пенсии и вынужденная работа может показать сумму, превышающую величину более высокой пенсии, но свидетельствовать о более низкой оценке себя...)
подходит для оценки показателей "Тип_занятости", "Количество_детей", ...

In [101]:
# --- Подготовка данных. Предварительная обработка необходимых столбцов  ---

# Чтение данных
data_necessary = pd.read_excel('../output_data/data_to_experiments.xlsx', sheet_name = 'data')

# Удаление первого столбца с номерами строк, "доставщихся по наследству" от первого файла
data_necessary.drop(data_necessary.columns[[0]], axis=1, inplace=True) 

# Объединение столбцов с пенсией и зарплатой
data_necessary.loc[data_necessary['Средняя_зарплата_месяц'] == '     .  ', 'Средняя_зарплата_месяц'] = 0
data_necessary.loc[data_necessary['Сумма_пенсии'] == '     .  ', 'Сумма_пенсии'] = 0
data_necessary.loc[data_necessary['Средняя_зарплата_месяц'] == 0, 'Средняя_зарплата_месяц'] = data_necessary['Средняя_зарплата_месяц'] + data_necessary['Сумма_пенсии']
data_necessary.rename(columns = dict(zip(['Средняя_зарплата_месяц'], ['Размер_зарплаты_пенсии'])), inplace = True) 
del data_necessary['Сумма_пенсии'] 

# Удаление строк с неуказанной зарплатой
data_necessary['Размер_зарплаты_пенсии'].replace(0, np.nan, inplace=True)
data_necessary.dropna(subset=['Размер_зарплаты_пенсии'], inplace=True)

# # Проверка пустых значений
# for title in list(data_necessary.columns.values):
#     data_necessary.loc[data_necessary[title] == '     .  ', title] = 0   
#     print(f'Поле {title}. Нулей: {data_necessary[title].isnull().sum()}.')
    
# Сохранение подготовленных данных в файл
data_necessary.to_excel('../output_data/ready_data.xlsx', sheet_name = 'data')

# Отображение данных по датафрейму
len(data_necessary)
# data_necessary

7851

In [ ]:
# Поскольку все поля, кроме признаки 'Размер_зарплаты_пенсии' являются категориальными, то нормализация значений не требуется